---

# 0. Imports

---

In [1]:
# Basic imports
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt

# Functional imports
import json
import pyarrow
from pickle import dump
import gc

# Model imports
from sklearn import preprocessing
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import f_classif, SelectKBest

pd.set_option('display.max_columns', None)

---

# 1. Data Collection

In [2]:
#df_2018 = pd.read_parquet("../data/raw/Combined_Flights_2018.parquet", engine="pyarrow")
#df_2019 = pd.read_parquet("../data/raw/Combined_Flights_2019.parquet", engine="pyarrow")
#df_2020 = pd.read_parquet("../data/raw/Combined_Flights_2020.parquet", engine="pyarrow")
#df_2021 = pd.read_parquet("../data/raw/Combined_Flights_2021.parquet", engine="pyarrow")
df = pd.read_parquet("../data/raw/Combined_Flights_2022.parquet", engine="pyarrow")


In [3]:
#data_frames = [df_2018, df_2019, df_2020, df_2021, df_2022]
#data_frames = [df_2021, df_2022]
#df = pd.concat(data_frames, ignore_index=True)

In [4]:
df.head(3)

,FlightDate,Airline,Origin,Dest,Cancelled,Diverted,CRSDepTime,DepTime,DepDelayMinutes,DepDelay,ArrTime,ArrDelayMinutes,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,Year,Quarter,Month,DayofMonth,DayOfWeek,Marketing_Airline_Network,Operated_or_Branded_Code_Share_Partners,DOT_ID_Marketing_Airline,IATA_Code_Marketing_Airline,Flight_Number_Marketing_Airline,Operating_Airline,DOT_ID_Operating_Airline,IATA_Code_Operating_Airline,Tail_Number,Flight_Number_Operating_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,DestCityName,DestState,DestStateFips,DestStateName,DestWac,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrDelay,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,DistanceGroup,DivAirportLandings
0,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",GJT,DEN,False,False,1133,1123.0,0.0,-10.0,1228.0,0.0,40.0,72.0,65.0,212.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4301,C5,20445,C5,N21144,4301,11921,1192102,31921,"Grand Junction, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1100-1159,17.0,1140.0,1220.0,8.0,1245,-17.0,0.0,-2.0,1200-1259,1,0
1,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",HRL,IAH,False,False,732,728.0,0.0,-4.0,848.0,0.0,55.0,77.0,80.0,295.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4299,C5,20445,C5,N16170,4299,12206,1220605,32206,"Harlingen/San Benito, TX",TX,48,Texas,74,12266,1226603,31453,"Houston, TX",TX,48,Texas,74,0.0,-1.0,0700-0759,16.0,744.0,839.0,9.0,849,-1.0,0.0,-1.0,0800-0859,2,0
2,2022-04-04,"Commutair Aka Champlain Enterprises, Inc.",DRO,DEN,False,False,1529,1514.0,0.0,-15.0,1636.0,0.0,47.0,70.0,82.0,251.0,2022,2,4,4,1,UA,UA_CODESHARE,19977,UA,4298,C5,20445,C5,N21144,4298,11413,1141307,30285,"Durango, CO",CO,8,Colorado,82,11292,1129202,30325,"Denver, CO",CO,8,Colorado,82,0.0,-1.0,1500-1559,21.0,1535.0,1622.0,14.0,1639,-3.0,0.0,-1.0,1600-1659,2,0


---

# 2 Exploration and Data cleaning

## Explicación de Columnas - Dataset de Vuelos

### Información del Vuelo
* **FlightDate**: Fecha del vuelo (año, mes y día)
* **Airline**: Código de aerolínea operadora
* **Origin**: Código IATA aeropuerto origen
* **Dest**: Código IATA aeropuerto destino

### Indicadores de Interrupción
* **Cancelled**: Vuelo cancelado (1=Sí, 0=No)
* **Diverted**: Vuelo desviado (1=Sí, 0=No)

### Horarios y Retrasos
#### Salida
* **CRSDepTime**: Hora programada salida
* **DepTime**: Hora real salida
* **DepDelayMinutes**: Minutos de retraso salida
* **DepDelay**: Retraso salida (negativo=anticipado)

#### Llegada
* **ArrTime**: Hora real llegada
* **ArrDelayMinutes**: Minutos retraso llegada 
* **ArrDelay**: Retraso llegada (negativo=anticipada)

### Duración
* **AirTime**: Minutos en aire
* **CRSElapsedTime**: Tiempo estimado total
* **ActualElapsedTime**: Tiempo real total

### Geografía y Tiempo
* **Distance**: Distancia en millas
* **Year**: Año del vuelo
* **Quarter**: Trimestre (1-4)
* **Month**: Mes (1-12)
* **DayofMonth**: Día del mes
* **DayOfWeek**: Día semana (1=Lunes, 7=Domingo)

### Información Aerolíneas
* **Marketing_Airline_Network**: Aerolínea vendedora
* **Operating_Airline**: Aerolínea operadora física
* **DOT_ID_Marketing_Airline**: ID DOT aerolínea marketing
* **IATA_Code_Marketing_Airline**: Código IATA aerolínea marketing
* **Flight_Number_Marketing_Airline**: Número vuelo marketing
* **DOT_ID_Operating_Airline**: ID DOT aerolínea operadora
* **IATA_Code_Operating_Airline**: Código IATA aerolínea operadora

### Aeronave
* **Tail_Number**: Matrícula única
* **Flight_Number_Operating_Airline**: Número vuelo operativo

### Aeropuerto Origen
* **OriginAirportID**: ID único
* **OriginAirportSeqID**: ID secuencia
* **OriginCityMarketID**: ID mercado ciudad
* **OriginCityName**: Nombre ciudad
* **OriginState**: Abreviación estado
* **OriginStateFips**: Código FIPS estado
* **OriginStateName**: Nombre estado
* **OriginWac**: Código WAC

### Aeropuerto Destino
* **DestAirportID**: ID único
* **DestAirportSeqID**: ID secuencia
* **DestCityMarketID**: ID mercado ciudad
* **DestCityName**: Nombre ciudad
* **DestState**: Abreviación estado
* **DestStateFips**: Código FIPS estado
* **DestStateName**: Nombre estado
* **DestWac**: Código WAC

### Indicadores Retraso Salida
* **DepDel15**: Retraso >15 min (1=Sí, 0=No)
* **DepartureDelayGroups**: Grupos de retraso
* **DepTimeBlk**: Bloque horario (ej: "0700-0759")

### Movimientos Aeronave
* **TaxiOut**: Minutos en pista antes despegue
* **WheelsOff**: Hora despegue
* **WheelsOn**: Hora aterrizaje
* **TaxiIn**: Minutos en pista post-aterrizaje

### Indicadores Retraso Llegada
* **CRSArrTime**: Hora programada llegada
* **ArrDel15**: Retraso >15 min (1=Sí, 0=No)
* **ArrivalDelayGroups**: Grupos de retraso
* **ArrTimeBlk**: Bloque horario llegada

### Otros Indicadores
* **DistanceGroup**: Grupos por distancia
* **DivAirportLandings**: Aterrizajes no programados

In [5]:
print('Our dataframe contains {} rows and it has {} features.'.format(len(df), df.shape[1]))

Our dataframe contains 4078318 rows and it has 61 features.


---

## 2.2 Eliminating Irrelevant Features and dealing with null values

Eliminaremos algunos features antes de continuar con nuestro EDA para ahorrar en recursos y tiempo

In [6]:
columnas_necesarias = [
   'DayOfWeek',
   'Month',
   'DayofMonth',
   'Airline',
   'OriginCityName',
   'DestCityName',
   'CRSDepTime',
   'DepTime',
   'DepDelay',
   'DepDel15'
]

In [7]:
df = df[columnas_necesarias]
df.head(3)

,DayOfWeek,Month,DayofMonth,Airline,OriginCityName,DestCityName,CRSDepTime,DepTime,DepDelay,DepDel15
0,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Grand Junction, CO","Denver, CO",1133,1123.0,-10.0,0.0
1,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Harlingen/San Benito, TX","Houston, TX",732,728.0,-4.0,0.0
2,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Durango, CO","Denver, CO",1529,1514.0,-15.0,0.0


In [8]:
# Date Transformation
import calendar

# 1 = Weekday, 0 = Weekend
Day_Week_Type = []
for value in df['DayOfWeek']:
    if value in (0,1,2,3,4):
        Day_Week_Type.append(1) # Weekday
    else:
        Day_Week_Type.append(0) # Weekend
df['WeekdayType'] = Day_Week_Type
df.head()

,DayOfWeek,Month,DayofMonth,Airline,OriginCityName,DestCityName,CRSDepTime,DepTime,DepDelay,DepDel15,WeekdayType
0,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Grand Junction, CO","Denver, CO",1133,1123.0,-10.0,0.0,1
1,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Harlingen/San Benito, TX","Houston, TX",732,728.0,-4.0,0.0,1
2,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Durango, CO","Denver, CO",1529,1514.0,-15.0,0.0,1
3,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Houston, TX","Gulfport/Biloxi, MS",1435,1430.0,-5.0,0.0,1
4,1,4,4,"Commutair Aka Champlain Enterprises, Inc.","Durango, CO","Denver, CO",1135,1135.0,0.0,0.0,1


In [9]:
df.shape

(4078318, 11)

In [10]:
'''def estratificar_datos(df, target_col='DepDelayMinutes', sample_size=1000000):
    """
    Realiza un muestreo estratificado manteniendo las proporciones de las variables.
    
    Parámetros:
    df: DataFrame original
    
    target_col: Nombre de la columna objetivo
    sample_size: Tamaño deseado de la muestra
    """
    
    # Crear bins para la variable objetivo (DepDelayMinutes)
    df['delay_category'] = pd.qcut(df[target_col], q=10, labels=False, duplicates='drop')
    
    # Crear una categoría combinada para la estratificación
    df['strata'] = df['delay_category'].astype(str) + '_' + \
                   df['Airline'].astype(str) + '_' + \
                   df['Quarter'].astype(str)
    
    # Calcular el tamaño proporcional para cada estrato
    proportions = df['strata'].value_counts(normalize=True)
    
    # Inicializar DataFrame vacío para la muestra
    stratified_sample = pd.DataFrame()
    
    # Realizar el muestreo estratificado
    for stratum in proportions.index:
        stratum_size = int(proportions[stratum] * sample_size)
        if stratum_size > 0:
            stratum_sample = df[df['strata'] == stratum].sample(
                n=min(stratum_size, len(df[df['strata'] == stratum])),
                random_state=42
            )
            stratified_sample = pd.concat([stratified_sample, stratum_sample])
    
    # Eliminar columnas auxiliares
    stratified_sample = stratified_sample.drop(['delay_category', 'strata'], axis=1)
    
    return stratified_sample

# Uso del código
# Asumiendo que tu DataFrame original se llama 'df'
# df = pd.read_csv('tu_archivo.csv')

# Crear muestra estratificada
muestra_estratificada = estratificar_datos(
    df,
    target_col='DepDelayMinutes',
    sample_size= 1450000  # Ajusta este número según tus necesidades
)

# Verificar las proporciones
def verificar_proporciones(df_original, df_muestra):
    """
    Compara las proporciones de las variables clave entre el DataFrame original y la muestra
    """
    variables_check = ['Airline', 'Quarter', 'DepDelayMinutes']
    
    for var in variables_check:
        if var == 'DepDelayMinutes':
            # Para variables numéricas, comparamos estadísticas descriptivas
            print(f"\nEstadísticas de {var}:")
            print("\nOriginal:")
            print(df_original[var].describe())
            print("\nMuestra:")
            print(df_muestra[var].describe())
        else:
            # Para variables categóricas, comparamos proporciones
            print(f"\nProporciones de {var}:")
            print("\nOriginal:")
            print(df_original[var].value_counts(normalize=True))
            print("\nMuestra:")
            print(df_muestra[var].value_counts(normalize=True))

# Verificar las proporciones
verificar_proporciones(df, muestra_estratificada)'''

'def estratificar_datos(df, target_col=\'DepDelayMinutes\', sample_size=1000000):\n    """\n    Realiza un muestreo estratificado manteniendo las proporciones de las variables.\n    \n    Parámetros:\n    df: DataFrame original\n    \n    target_col: Nombre de la columna objetivo\n    sample_size: Tamaño deseado de la muestra\n    """\n    \n    # Crear bins para la variable objetivo (DepDelayMinutes)\n    df[\'delay_category\'] = pd.qcut(df[target_col], q=10, labels=False, duplicates=\'drop\')\n    \n    # Crear una categoría combinada para la estratificación\n    df[\'strata\'] = df[\'delay_category\'].astype(str) + \'_\' +                    df[\'Airline\'].astype(str) + \'_\' +                    df[\'Quarter\'].astype(str)\n    \n    # Calcular el tamaño proporcional para cada estrato\n    proportions = df[\'strata\'].value_counts(normalize=True)\n    \n    # Inicializar DataFrame vacío para la muestra\n    stratified_sample = pd.DataFrame()\n    \n    # Realizar el muestreo estra

### Análisis de Variables Seleccionadas para Predicción de Retrasos en Vuelos

### Variables Temporales
- `Month`, `DayOfWeek`: Capturan patrones estacionales y semanales
- `CRSDepTime`: Hora programada, afecta probabilidad de retrasos
- `DepTime`: Hora real de salida, necesaria para validación

### Variables Operativas
- `Airline`: Diferentes aerolíneas tienen distintos índices de puntualidad
- `Distance`: Factor crítico que afecta complejidad operativa
- `DepDelayMinutes`: Variable objetivo para predicción

### Variables Geográficas
- `Origin`, `Dest`: Identificadores de aeropuertos
- `OriginAirportID`: Identificador único para análisis consistente
- `OriginStateName`, `DestStateName`: Factores climatológicos regionales
- `DestCityName`: Patrones específicos de destinos urbanos

### Variables Excluidas y Razones
- Variables redundantes (ej: `DepDelay`, similar a `DepDelayMinutes`)
- Datos post-evento (ej: `ArrTime`, `AirTime`)
- Información detallada no predictiva (ej: `TailNumber`)
- Variables derivadas calculables desde las seleccionadas

## 2.3 Analisis de nuestra dataframe

In [11]:
df.describe()

,DayOfWeek,Month,DayofMonth,CRSDepTime,DepTime,DepDelay,DepDel15,WeekdayType
count,4.078318e+06,4.078318e+06,4.078318e+06,4.078318e+06,3.957885e+06,3.957823e+06,3.957823e+06,4.078318e+06
mean,4.005566e+00,4.083803e+00,1.571131e+01,1.329587e+03,1.334374e+03,1.309049e+01,2.184625e-01,5.708395e-01
std,2.007050e+00,1.998918e+00,8.760122e+00,4.904801e+02,5.056219e+02,5.332016e+01,4.132030e-01,4.949564e-01
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,-7.800000e+01,0.000000e+00,0.000000e+00
25%,2.000000e+00,2.000000e+00,8.000000e+00,9.140000e+02,9.170000e+02,-5.000000e+00,0.000000e+00,0.000000e+00
50%,4.000000e+00,4.000000e+00,1.600000e+01,1.320000e+03,1.325000e+03,-2.000000e+00,0.000000e+00,1.000000e+00
75%,6.000000e+00,6.000000e+00,2.300000e+01,1.735000e+03,1.744000e+03,1.100000e+01,0.000000e+00,1.000000e+00
max,7.000000e+00,7.000000e+00,3.100000e+01,2.359000e+03,2.400000e+03,7.223000e+03,1.000000e+00,1.000000e+00


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4078318 entries, 0 to 590541
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   DayOfWeek       int64  
 1   Month           int64  
 2   DayofMonth      int64  
 3   Airline         object 
 4   OriginCityName  object 
 5   DestCityName    object 
 6   CRSDepTime      int64  
 7   DepTime         float64
 8   DepDelay        float64
 9   DepDel15        float64
 10  WeekdayType     int64  
dtypes: float64(3), int64(5), object(3)
memory usage: 373.4+ MB


***Coclusion:***

Nuestro data frame contiene los siguientes tipos de datos:

- ``Float``     --> 3
- ``Integer``   --> 4
- ``String``    --> 7



## 2.4 Buscando duplicados, valores vacios y lidiando con ellos

In [13]:
df.duplicated().sum()

434

In [14]:
df = df.drop_duplicates()

In [15]:
df.duplicated().sum()

0

In [16]:
df.isna().sum()[df.isna().sum()>0]

DepTime     120344
DepDelay    120406
DepDel15    120406
dtype: int64

In [17]:
df = df.dropna()

In [18]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

***Conclusion:***

De aproximadamente 29M de filas se encontrarons solo 573 filas duplicadas y 762978 de valores nulos

Sabiendo estos numeros lo que haremos seremos eliminarlos para que no afecte en el analisis de datos. Esta decision fue tomada considerando que al ser un porcentaje bajo de data no afectara al momento del analisis

---

# 3. Casting

## 3.1 Dividing our dataset into categorical, numerical and temporal 

In [19]:
# Understanding how many type of features we have
df.dtypes.unique()

array([dtype('int64'), dtype('O'), dtype('float64')], dtype=object)

In [20]:
num = ["float64", "int64"]
cat = ['O']

df_num = df.select_dtypes(num)
df_cat = df.select_dtypes(cat)

In [21]:
#Numerical Dataframe
df_num.head(3)

,DayOfWeek,Month,DayofMonth,CRSDepTime,DepTime,DepDelay,DepDel15,WeekdayType
0,1,4,4,1133,1123.0,-10.0,0.0,1
1,1,4,4,732,728.0,-4.0,0.0,1
2,1,4,4,1529,1514.0,-15.0,0.0,1


In [22]:
#Categorical Dataframe
df_cat.head(3)

,Airline,OriginCityName,DestCityName
0,"Commutair Aka Champlain Enterprises, Inc.","Grand Junction, CO","Denver, CO"
1,"Commutair Aka Champlain Enterprises, Inc.","Harlingen/San Benito, TX","Houston, TX"
2,"Commutair Aka Champlain Enterprises, Inc.","Durango, CO","Denver, CO"


---

# 4. Univariated Analysis

## 4.1 Categorical Variable Analysis 

In [23]:
df_cat.head(3)

,Airline,OriginCityName,DestCityName
0,"Commutair Aka Champlain Enterprises, Inc.","Grand Junction, CO","Denver, CO"
1,"Commutair Aka Champlain Enterprises, Inc.","Harlingen/San Benito, TX","Houston, TX"
2,"Commutair Aka Champlain Enterprises, Inc.","Durango, CO","Denver, CO"


In [24]:
'''fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(10,20))
sns.set_palette("bright")

# Airline
airline_counts = df_cat["Airline"].value_counts()
airlines_filtered = airline_counts[airline_counts >= 50000]
sns.countplot(ax=ax[0], 
              data=df_cat[df_cat["Airline"].isin(airlines_filtered.index)],
              y="Airline", 
              order=airlines_filtered.index,
              hue="Airline")

# Origin State
origin_state_counts = df_cat["OriginStateName"].value_counts()
origin_states_filtered = origin_state_counts[origin_state_counts >= 50000]
sns.countplot(ax=ax[1], 
              data=df_cat[df_cat["OriginStateName"].isin(origin_states_filtered.index)],
              y="OriginStateName", 
              order=origin_states_filtered.index,
              hue="OriginStateName")

# Destination State
dest_state_counts = df_cat["DestStateName"].value_counts()
dest_states_filtered = dest_state_counts[dest_state_counts >= 50000]
sns.countplot(ax=ax[2], 
              data=df_cat[df_cat["DestStateName"].isin(dest_states_filtered.index)],
              y="DestStateName", 
              order=dest_states_filtered.index,
              hue="DestStateName")

# Origin City
origin_city_counts = df_cat["OriginCityName"].value_counts()
origin_cities_filtered = origin_city_counts[origin_city_counts >= 50000]
sns.countplot(ax=ax[3], 
              data=df_cat[df_cat["OriginCityName"].isin(origin_cities_filtered.index)],
              y="OriginCityName", 
              order=origin_cities_filtered.index,
              hue="OriginCityName")

# Destination City
dest_city_counts = df_cat["DestCityName"].value_counts()
dest_cities_filtered = dest_city_counts[dest_city_counts >= 50000]
sns.countplot(ax=ax[4], 
              data=df_cat[df_cat["DestCityName"].isin(dest_cities_filtered.index)],
              y="DestCityName", 
              order=dest_cities_filtered.index,
              hue="DestCityName")

# Títulos
ax[0].set_title("Airline Travel Numbers (≥50,000)", fontsize=14, fontweight='bold')
ax[1].set_title("Travel numbers from this state (≥50,000)", fontsize=14, fontweight='bold')
ax[2].set_title("Travel numbers to this state (≥50,000)", fontsize=14, fontweight='bold')
ax[3].set_title("Travel numbers from this city (≥50,000)", fontsize=14, fontweight='bold')
ax[4].set_title("Travel numbers to this city (≥50,000)", fontsize=14, fontweight='bold')

plt.tight_layout()
fig.subplots_adjust(hspace=0.25)
plt.show()'''

'fig, ax = plt.subplots(nrows=5, ncols=1, figsize=(10,20))\nsns.set_palette("bright")\n\n# Airline\nairline_counts = df_cat["Airline"].value_counts()\nairlines_filtered = airline_counts[airline_counts >= 50000]\nsns.countplot(ax=ax[0], \n              data=df_cat[df_cat["Airline"].isin(airlines_filtered.index)],\n              y="Airline", \n              order=airlines_filtered.index,\n              hue="Airline")\n\n# Origin State\norigin_state_counts = df_cat["OriginStateName"].value_counts()\norigin_states_filtered = origin_state_counts[origin_state_counts >= 50000]\nsns.countplot(ax=ax[1], \n              data=df_cat[df_cat["OriginStateName"].isin(origin_states_filtered.index)],\n              y="OriginStateName", \n              order=origin_states_filtered.index,\n              hue="OriginStateName")\n\n# Destination State\ndest_state_counts = df_cat["DestStateName"].value_counts()\ndest_states_filtered = dest_state_counts[dest_state_counts >= 50000]\nsns.countplot(ax=ax[2], \n 

## 4.2 Numerical Variable Analysis

In [25]:
df_num.head()

,DayOfWeek,Month,DayofMonth,CRSDepTime,DepTime,DepDelay,DepDel15,WeekdayType
0,1,4,4,1133,1123.0,-10.0,0.0,1
1,1,4,4,732,728.0,-4.0,0.0,1
2,1,4,4,1529,1514.0,-15.0,0.0,1
3,1,4,4,1435,1430.0,-5.0,0.0,1
4,1,4,4,1135,1135.0,0.0,0.0,1


In [26]:
'''fig, ax = plt.subplots(8, 2, figsize=(10,12), gridspec_kw={"height_ratios":[6,1,6,1,6,1,6,1]})
sns.reset_defaults()

#Row_1
sns.histplot(ax = ax[0,0], data=df_num, x="DayOfWeek")
sns.boxplot(ax = ax[1,0], data=df_num, x="DayOfWeek")
sns.histplot(ax = ax[0,1], data=df_num, x="Month").set(ylabel=None)
sns.boxplot(ax = ax[1,1], data=df_num, x="Month")

#Row_2
sns.histplot(ax = ax[2,0], data=df_num, x="Quarter")
sns.boxplot(ax = ax[3,0], data=df_num, x="Quarter")
sns.histplot(ax = ax[2,1], data=df_num, x="DayofMonth").set(ylabel=None)
sns.boxplot(ax = ax[3,1], data=df_num, x="DayofMonth")

#Row_3
sns.histplot(ax = ax[4,0], data=df_num[(df_num["DepDelayMinutes"] >= 1) & (df_num["DepDelayMinutes"] <= 60)], x="DepDelayMinutes")
sns.boxplot(ax = ax[5,0], data=df_num, x="DepDelayMinutes")
sns.histplot(ax = ax[4,1], data=df_num, x="DepTime").set(ylabel=None)
sns.boxplot(ax = ax[5,1], data=df_num, x="DepTime")


#Row_4
sns.histplot(ax = ax[6,0], data=df_num, x="CRSDepTime")
sns.boxplot(ax = ax[7,0], data=df_num, x="CRSDepTime")
sns.histplot(ax = ax[6,1], data=df_num[df["Distance"] <= 2000], x="Distance")
sns.boxplot(ax = ax[7,1], data=df_num, x="Distance")

plt.tight_layout()
fig.subplots_adjust(hspace=1)
plt.show()'''

'fig, ax = plt.subplots(8, 2, figsize=(10,12), gridspec_kw={"height_ratios":[6,1,6,1,6,1,6,1]})\nsns.reset_defaults()\n\n#Row_1\nsns.histplot(ax = ax[0,0], data=df_num, x="DayOfWeek")\nsns.boxplot(ax = ax[1,0], data=df_num, x="DayOfWeek")\nsns.histplot(ax = ax[0,1], data=df_num, x="Month").set(ylabel=None)\nsns.boxplot(ax = ax[1,1], data=df_num, x="Month")\n\n#Row_2\nsns.histplot(ax = ax[2,0], data=df_num, x="Quarter")\nsns.boxplot(ax = ax[3,0], data=df_num, x="Quarter")\nsns.histplot(ax = ax[2,1], data=df_num, x="DayofMonth").set(ylabel=None)\nsns.boxplot(ax = ax[3,1], data=df_num, x="DayofMonth")\n\n#Row_3\nsns.histplot(ax = ax[4,0], data=df_num[(df_num["DepDelayMinutes"] >= 1) & (df_num["DepDelayMinutes"] <= 60)], x="DepDelayMinutes")\nsns.boxplot(ax = ax[5,0], data=df_num, x="DepDelayMinutes")\nsns.histplot(ax = ax[4,1], data=df_num, x="DepTime").set(ylabel=None)\nsns.boxplot(ax = ax[5,1], data=df_num, x="DepTime")\n\n\n#Row_4\nsns.histplot(ax = ax[6,0], data=df_num, x="CRSDepTime"

## 4.3 Analysis Categorical - Categorical

In [27]:
'''fig, axis = plt.subplots(3, 1, figsize = (10, 20))

sns.barplot(ax= axis[0], data=df, y='Airline', x='DepDelayMinutes')
sns.barplot(ax= axis[1], data=df, y='OriginStateName', x='DepDelayMinutes')
sns.barplot(ax= axis[2], data=df, y='DestStateName', x='DepDelayMinutes')

axis[0].set_title("Promedio de Delays por Aerolínea", fontsize=14, fontweight='bold')
axis[1].set_title("Promedio de Delays por lugar de origen", fontsize=14, fontweight='bold')
axis[2].set_title("Promedio de Delays por lugar de llegada", fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()'''

'fig, axis = plt.subplots(3, 1, figsize = (10, 20))\n\nsns.barplot(ax= axis[0], data=df, y=\'Airline\', x=\'DepDelayMinutes\')\nsns.barplot(ax= axis[1], data=df, y=\'OriginStateName\', x=\'DepDelayMinutes\')\nsns.barplot(ax= axis[2], data=df, y=\'DestStateName\', x=\'DepDelayMinutes\')\n\naxis[0].set_title("Promedio de Delays por Aerolínea", fontsize=14, fontweight=\'bold\')\naxis[1].set_title("Promedio de Delays por lugar de origen", fontsize=14, fontweight=\'bold\')\naxis[2].set_title("Promedio de Delays por lugar de llegada", fontsize=14, fontweight=\'bold\')\n\nplt.tight_layout()\nplt.show()'

## 4.4 Analysis Numerical - Numerical


In [28]:
df_num.columns

Index(['DayOfWeek', 'Month', 'DayofMonth', 'CRSDepTime', 'DepTime', 'DepDelay',
       'DepDel15', 'WeekdayType'],
      dtype='object')

In [29]:
'''# Tomar muestra y pre-calcular correlaciones
df_sample = df_num.sample(frac=0.5, random_state=42)
corr_matrix = df_num[['DayOfWeek', 'Month', 'Quarter', 'DayofMonth', 
                      'DepDelayMinutes', 'DepTime', 'CRSDepTime', 'Distance']].corr()

fig, axis = plt.subplots(4, 4, figsize=(20, 20))

# DayOfWeek
sns.regplot(ax=axis[0, 0], data=df_sample, x="DayOfWeek", y="DepDelayMinutes", 
            scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DayOfWeek"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 0], cbar=False)

# Month
sns.regplot(ax=axis[0, 1], data=df_sample, x="Month", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[2], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Month"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 1])

# Quarter
sns.regplot(ax=axis[0, 2], data=df_sample, x="Quarter", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[3], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Quarter"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 2])

# DayofMonth
sns.regplot(ax=axis[0, 3], data=df_sample, x="DayofMonth", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[4], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DayofMonth"]].T, 
            annot=True, fmt=".2f", ax=axis[1, 3])

# DepTime
sns.regplot(ax=axis[2, 0], data=df_sample, x="DepTime", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[5], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DepTime"]].T, 
            annot=True, fmt=".2f", ax=axis[3, 0])

# CRSDepTime
sns.regplot(ax=axis[2, 1], data=df_sample, x="CRSDepTime", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[6], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["CRSDepTime"]].T, 
            annot=True, fmt=".2f", ax=axis[3, 1])

# Distance
sns.regplot(ax=axis[2, 2], data=df_sample, x="Distance", y="DepDelayMinutes", 
            color=sns.color_palette("colorblind")[7], scatter_kws={'alpha':0.3}, n_boot=100)
sns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Distance"]].T, 
            annot=True, fmt=".2f", ax=axis[3, 2])

# Ocultar los subplots vacíos
axis[2, 3].set_visible(False)
axis[3, 3].set_visible(False)

# Ajustar títulos
axis[0, 0].set_title("Day of Week vs Delay", fontsize=12)
axis[0, 1].set_title("Month vs Delay", fontsize=12)
axis[0, 2].set_title("Quarter vs Delay", fontsize=12)
axis[0, 3].set_title("Day of Month vs Delay", fontsize=12)
axis[2, 0].set_title("Departure Time vs Delay", fontsize=12)
axis[2, 1].set_title("Scheduled Departure Time vs Delay", fontsize=12)
axis[2, 2].set_title("Distance vs Delay", fontsize=12)

# Ajustar el diseño
plt.tight_layout()
plt.show()'''

'# Tomar muestra y pre-calcular correlaciones\ndf_sample = df_num.sample(frac=0.5, random_state=42)\ncorr_matrix = df_num[[\'DayOfWeek\', \'Month\', \'Quarter\', \'DayofMonth\', \n                      \'DepDelayMinutes\', \'DepTime\', \'CRSDepTime\', \'Distance\']].corr()\n\nfig, axis = plt.subplots(4, 4, figsize=(20, 20))\n\n# DayOfWeek\nsns.regplot(ax=axis[0, 0], data=df_sample, x="DayOfWeek", y="DepDelayMinutes", \n            scatter_kws={\'alpha\':0.3}, n_boot=100)\nsns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["DayOfWeek"]].T, \n            annot=True, fmt=".2f", ax=axis[1, 0], cbar=False)\n\n# Month\nsns.regplot(ax=axis[0, 1], data=df_sample, x="Month", y="DepDelayMinutes", \n            color=sns.color_palette("colorblind")[2], scatter_kws={\'alpha\':0.3}, n_boot=100)\nsns.heatmap(corr_matrix.loc[["DepDelayMinutes"], ["Month"]].T, \n            annot=True, fmt=".2f", ax=axis[1, 1])\n\n# Quarter\nsns.regplot(ax=axis[0, 2], data=df_sample, x="Quarter", y="DepDelayMinutes", \

---

# 5. Multivariate Analysis

## 5.1 Encoding categorical variables


In [30]:
'''Label_Encoder = preprocessing.LabelEncoder()



df['Airline'] = Label_Encoder.fit_transform(df['Airline'])
df['OriginCityName'] = Label_Encoder.fit_transform(df['OriginCityName'])
df['DestCityName'] = Label_Encoder.fit_transform(df['DestCityName'])
df.dtypes.sort_values()
'''


"Label_Encoder = preprocessing.LabelEncoder()\n\n\n\ndf['Airline'] = Label_Encoder.fit_transform(df['Airline'])\ndf['OriginCityName'] = Label_Encoder.fit_transform(df['OriginCityName'])\ndf['DestCityName'] = Label_Encoder.fit_transform(df['DestCityName'])\ndf.dtypes.sort_values()\n"

In [31]:
for column in df_cat.columns:
    unique_values = list(df_cat[column].unique())
    globals()[f"{column}_enc"] = dict(zip(unique_values, range(len(unique_values))))

    json.dump(globals()[f"{column}_enc"], open(f'../data/interim/enc_{column}.json', 'w'))

In [32]:
# Replacing the values in our categorical features to our encoded values (numerical)
for column in df_cat.columns:
    df[column] = df[column].map(json.load(open(f'../data/interim/enc_{column}.json')))

In [33]:
df.head(3)

,DayOfWeek,Month,DayofMonth,Airline,OriginCityName,DestCityName,CRSDepTime,DepTime,DepDelay,DepDel15,WeekdayType
0,1,4,4,0,0,0,1133,1123.0,-10.0,0.0,1
1,1,4,4,0,1,1,732,728.0,-4.0,0.0,1
2,1,4,4,0,2,0,1529,1514.0,-15.0,0.0,1


## 5.2 Numerical - Categorical analysis (Correlational Analysis)

In [34]:
'''sns.pairplot(data=df_enc)
plt.show()'''

'sns.pairplot(data=df_enc)\nplt.show()'

In [35]:
'''plt.figure(figsize=(20,16))
sns.heatmap(df_enc.corr().round(2), 
            annot=True,
            mask=np.triu(df_enc.corr()),
            cmap='RdBu',
            center=0,
            square=True,
            fmt='.2f',
            annot_kws={'size': 8},  
            cbar_kws={'shrink': .8}
            )
plt.show()'''

"plt.figure(figsize=(20,16))\nsns.heatmap(df_enc.corr().round(2), \n            annot=True,\n            mask=np.triu(df_enc.corr()),\n            cmap='RdBu',\n            center=0,\n            square=True,\n            fmt='.2f',\n            annot_kws={'size': 8},  \n            cbar_kws={'shrink': .8}\n            )\nplt.show()"

---

# 6. Feature Engineering

## 6.1 Creating new features 

### - Variable de franja horaria
Esto lo hacemos debido a que usualmente los vuelos son retrasados por franjas horarias. Y si se percatan no agregaremos los minutos exactos porque esto puede hacer que los datos esten muy granulados y podria generar un Overfitting del modelo

In [36]:
'''def convert_military_to_quarter_hour(time):
    if pd.isna(time):
        return np.nan
    
    time_str = str(int(time)).zfill(4)
    hour = int(time_str[:2])
    minutes = int(time_str[2:])
    
    # Convertir minutos a cuartos de hora
    quarter = (minutes // 15) * 0.25
    
    # Combinar hora con cuarto de hora
    return hour + quarter
    # Ejemplos:
    # 14:00 -> 14.0
    # 14:15 -> 14.25
    # 14:30 -> 14.5
    # 14:45 -> 14.75

# Aplicar la función
df['HourBlock'] = df['DepTime'].apply(convert_military_to_quarter_hour)'''

"def convert_military_to_quarter_hour(time):\n    if pd.isna(time):\n        return np.nan\n    \n    time_str = str(int(time)).zfill(4)\n    hour = int(time_str[:2])\n    minutes = int(time_str[2:])\n    \n    # Convertir minutos a cuartos de hora\n    quarter = (minutes // 15) * 0.25\n    \n    # Combinar hora con cuarto de hora\n    return hour + quarter\n    # Ejemplos:\n    # 14:00 -> 14.0\n    # 14:15 -> 14.25\n    # 14:30 -> 14.5\n    # 14:45 -> 14.75\n\n# Aplicar la función\ndf['HourBlock'] = df['DepTime'].apply(convert_military_to_quarter_hour)"

In [37]:
'''df_enc["HourBlock"].dtype'''

'df_enc["HourBlock"].dtype'

In [38]:
'''df_enc.head()'''

'df_enc.head()'

### Convertir mis float64 y int64 a float32 e int32

## 6.2 Outlier Analysis

In [39]:
continuos_f = list(df.select_dtypes('float64'))

In [40]:
'''continuos_f = list(df_enc.select_dtypes('float64'))

fig, ax = plt.subplots(5,3, figsize=(10, 20))

row, col = 0, 0
for each in df_enc.columns:
    sns.boxplot(ax = ax[row,col], data = df_enc, x=each).tick_params(labelsize=2)
    col += 1
    if col == 3: 
        row += 1 
        col = 0

plt.tight_layout()
plt.show()'''

"continuos_f = list(df_enc.select_dtypes('float64'))\n\nfig, ax = plt.subplots(5,3, figsize=(10, 20))\n\nrow, col = 0, 0\nfor each in df_enc.columns:\n    sns.boxplot(ax = ax[row,col], data = df_enc, x=each).tick_params(labelsize=2)\n    col += 1\n    if col == 3: \n        row += 1 \n        col = 0\n\nplt.tight_layout()\nplt.show()"

In [41]:
'''def remove_datetime_columns(df):
    return df.select_dtypes(exclude=['datetime64[ns]'])'''

"def remove_datetime_columns(df):\n    return df.select_dtypes(exclude=['datetime64[ns]'])"

In [42]:
df_no = df.copy()

In [43]:
def remove_outliers(x, feature_name, allow_neg=True):
    q1, q3 = x.quantile([0.25, 0.75])
    iqr = q3 - q1
    upper_lim = q3 + (iqr*1.5)
    lower_lim = q1 - (iqr*1.5) if allow_neg else max(0, q1 - (iqr * 1.5))

    x = x.apply(lambda x: upper_lim if (x > upper_lim) else (lower_lim if (x < lower_lim) else x))

    filename = f'../data/interim/outliers_lims_{feature_name}.json'
    json.dump({'upper_lim': upper_lim, 'lower_lim': lower_lim}, open(filename, 'w'))

    return x

In [44]:
f_outliers_with_neg = []
f_outliers_no_neg = []

for each in continuos_f:
    if df[each].min() < 0:
        f_outliers_with_neg.append(each)
    f_outliers_no_neg.append(each)

for feature in f_outliers_with_neg:
    df_no[feature] = remove_outliers(df_no[feature], feature)

for feature in f_outliers_no_neg:
    try:
        df_no[feature] = remove_outliers(df_no[feature], feature, allow_neg=False)
    except:
        continue

In [45]:
df.isna().sum()[df.isna().sum()>0]

Series([], dtype: int64)

In [46]:
'''remove_datetime_columns(df_enc)
remove_datetime_columns(df_enc_no)'''

'remove_datetime_columns(df_enc)\nremove_datetime_columns(df_enc_no)'

In [47]:
def split(target, df, test_size=0.2, random_state=42):
    X = df.drop(columns=target)
    y = df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state,stratify=y)
    return X_train, X_test, y_train, y_test

In [48]:
# Split original dataframe 
X_train_with_outliers, X_test_with_outliers, y_train, y_test = split('DepDelay', df)
X_train_without_outliers, X_test_without_outliers, _, _ = split('DepDelay', df_no)

X_train_with_outliers.to_csv('../data/processed/X_train_with_outliers.csv', index=False)
X_test_with_outliers.to_csv('../data/processed/X_test_with_outliers.csv', index=False)
X_train_without_outliers.to_csv('../data/processed/X_train_without_outliers.csv', index=False)
X_test_without_outliers.to_csv('../data/processed/X_test_without_outliers.csv', index=False)

y_train.to_csv('../data/processed/y_train.csv', index=False)
y_test.to_csv('../data/processed/y_test.csv', index=False)

In [49]:
X_test_with_outliers.head()

,DayOfWeek,Month,DayofMonth,Airline,OriginCityName,DestCityName,CRSDepTime,DepTime,DepDelay,WeekdayType
205495,4,4,28,7,84,32,1934,1931.0,-3.0,1
530908,6,1,15,20,107,0,1159,1202.0,3.0,0
267960,4,1,13,5,84,130,600,549.0,-11.0,1
217535,2,5,24,7,158,25,1236,1236.0,0.0,1
101341,5,1,21,16,158,70,1621,1617.0,-4.0,0


In [50]:
'''def kselection(X_train, X_test, y_train, k, reference: str):
    selection_model = SelectKBest(f_classif, k=k)
    selection_model.fit(X_train, y_train)
    cols = selection_model.get_support()

    X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=X_train.columns.values[cols]) 
    X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=X_test.columns.values[cols])

    dump(selection_model, open(f'../data/processed/selection_model_{reference}.sav', 'wb'))

    return X_train_sel, X_test_sel'''

"def kselection(X_train, X_test, y_train, k, reference: str):\n    selection_model = SelectKBest(f_classif, k=k)\n    selection_model.fit(X_train, y_train)\n    cols = selection_model.get_support()\n\n    X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns=X_train.columns.values[cols]) \n    X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns=X_test.columns.values[cols])\n\n    dump(selection_model, open(f'../data/processed/selection_model_{reference}.sav', 'wb'))\n\n    return X_train_sel, X_test_sel"

In [51]:
'''X_train_with_outliers_sel, X_test_with_outliers_sel = kselection(X_train_with_outliers, X_test_with_outliers, y_train, 'all', 'with_outliers')
X_train_without_outliers_sel, X_test_without_outliers_sel = kselection(X_train_without_outliers, X_test_without_outliers, y_train, 'all', 'without_outliers')'''

"X_train_with_outliers_sel, X_test_with_outliers_sel = kselection(X_train_with_outliers, X_test_with_outliers, y_train, 'all', 'with_outliers')\nX_train_without_outliers_sel, X_test_without_outliers_sel = kselection(X_train_without_outliers, X_test_without_outliers, y_train, 'all', 'without_outliers')"

In [52]:
dfs_train = {
    'X_train_with_outliers': X_train_with_outliers,
    'X_train_without_outliers': X_train_without_outliers
}

dfs_test = {
    'X_test_with_outliers': X_test_with_outliers,
    'X_test_without_outliers': X_test_without_outliers   
}

for name, df in dfs_train.items():
    df.to_csv(f"../data/processed/{name}.csv", index=False)

for name, df in dfs_test.items(): 
    df.to_csv(f'../data/processed/{name}.csv', index=False)

In [53]:
train, test = [], []

for name, df in dfs_train.items():
    train.append(df)
for name, df in dfs_test.items():
    test.append(df)

In [54]:
X_train_with_outliers.columns

Index(['DayOfWeek', 'Month', 'DayofMonth', 'Airline', 'OriginCityName',
       'DestCityName', 'CRSDepTime', 'DepTime', 'DepDelay', 'WeekdayType'],
      dtype='object')

In [55]:
X_train_with_outliers.head()

,DayOfWeek,Month,DayofMonth,Airline,OriginCityName,DestCityName,CRSDepTime,DepTime,DepDelay,WeekdayType
366770,6,2,26,18,9,66,1344,1337.0,-7.0,0
571205,2,6,28,10,148,149,1515,1519.0,4.0,1
177591,1,2,21,7,124,104,821,924.0,63.0,1
559550,7,3,6,3,140,162,2259,2320.0,21.0,0
197065,2,3,29,7,131,108,2114,2110.0,-4.0,1


In [72]:
# Importación de librerías necesarias
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

In [76]:
model = XGBRegressor(
    objective='reg:squarederror',  # Para problemas de regresión
    n_estimators=100,              # Número de árboles
    max_depth=6,                   # Profundidad máxima de cada árbol
    learning_rate=0.1,             # Tasa de aprendizaje
    random_state=42,               # Para reproducibilidad
    eval_metric='rmse'             # Métrica para evaluar el rendimiento durante el entrenamiento
)

# Entrenamos el modelo
model.fit(
    train[0], y_train,
    eval_set=[(test[0], y_test)],
    verbose=True
)

[0]	validation_0-rmse:0.37189
[1]	validation_0-rmse:0.33470
[2]	validation_0-rmse:0.30123
[3]	validation_0-rmse:0.27111
[4]	validation_0-rmse:0.24400
[5]	validation_0-rmse:0.21960
[6]	validation_0-rmse:0.19764
[7]	validation_0-rmse:0.17788
[8]	validation_0-rmse:0.16009
[9]	validation_0-rmse:0.14408
[10]	validation_0-rmse:0.12967
[11]	validation_0-rmse:0.11670
[12]	validation_0-rmse:0.10503
[13]	validation_0-rmse:0.09453
[14]	validation_0-rmse:0.08508
[15]	validation_0-rmse:0.07657
[16]	validation_0-rmse:0.06891
[17]	validation_0-rmse:0.06202
[18]	validation_0-rmse:0.05582
[19]	validation_0-rmse:0.05024
[20]	validation_0-rmse:0.04521
[21]	validation_0-rmse:0.04069
[22]	validation_0-rmse:0.03662
[23]	validation_0-rmse:0.03296
[24]	validation_0-rmse:0.02966
[25]	validation_0-rmse:0.02670
[26]	validation_0-rmse:0.02403
[27]	validation_0-rmse:0.02163
[28]	validation_0-rmse:0.01946
[29]	validation_0-rmse:0.01752
[30]	validation_0-rmse:0.01577
[31]	validation_0-rmse:0.01419
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=6, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [78]:
# Evaluación del modelo
# Realizamos predicciones en el conjunto de prueba
y_pred = model.predict(test[0])

# Calculamos las métricas de rendimiento
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Métricas de Rendimiento:")
print(f"Error Cuadrático Medio (MSE): {mse:.2f}")
print(f"Raíz del Error Cuadrático Medio (RMSE): {rmse:.2f}")
print(f"Error Absoluto Medio (MAE): {mae:.2f}")
print(f"Coeficiente de Determinación (R²): {r2:.2f}")

Métricas de Rendimiento:
Error Cuadrático Medio (MSE): 0.00
Raíz del Error Cuadrático Medio (RMSE): 0.00
Error Absoluto Medio (MAE): 0.00
Coeficiente de Determinación (R²): 1.00


In [122]:
# Verificar que el modelo está prediciendo valores diferentes en el conjunto de prueba original
y_test_pred = best_model.predict(test[best_ind])
print("Muestra de predicciones en el conjunto de prueba:")
print(y_test_pred[:10])  # Primeras 10 predicciones
print(f"\nRango de predicciones:")
print(f"Mínimo: {min(y_test_pred)}")
print(f"Máximo: {max(y_test_pred)}")
print(f"Media: {np.mean(y_test_pred)}")

Muestra de predicciones en el conjunto de prueba:
[1.7848033 2.7839298 3.9961362 2.863127  2.4830184 1.7073576 3.1245356
 2.4707952 2.0766022 1.3044893]

Rango de predicciones:
Mínimo: -3.222576141357422
Máximo: 1527.4962158203125
Media: 14.079286575317383


In [123]:
# Crear casos con valores extremos
casos_extremos = pd.DataFrame({
    'DayOfWeek':       [1.0, 7.0],    
    'Month':           [1.0, 12.0],    
    'Quarter':         [1.0, 4.0],    
    'DayofMonth':      [1.0, 31.0],    
    'DepTime':         [0.0, 2359.0], 
    'CRSDepTime':      [0.0, 2359.0],
    'Distance':        [100.0, 3000.0],
    'Airline':         [min(X_train_with_outliers['Airline']), max(X_train_with_outliers['Airline'])],
    'OriginCityName':  [min(X_train_with_outliers['OriginCityName']), max(X_train_with_outliers['OriginCityName'])],
    'DestCityName':    [min(X_train_with_outliers['DestCityName']), max(X_train_with_outliers['DestCityName'])],
})

predicciones_extremas = best_model.predict(casos_extremos)
print("\nPredicciones con valores extremos:")
for i, pred in enumerate(predicciones_extremas):
    print(f"Caso extremo {i+1}: {format_delay_time(pred)}")


Predicciones con valores extremos:
Caso extremo 1: 3min
Caso extremo 2: 31min


In [124]:
casos_prueba = pd.DataFrame({
    # 1: Peor escenario - Vuelo en temporada alta, viernes por la tarde
    # 2: Mejor escenario - Vuelo temprano, día entre semana
    # 3: Caso típico - Vuelo de media distancia en horario normal
    # 4: Caso de alto riesgo - Último vuelo del día
    'DayOfWeek':       [5.0,   2.0,    3.0,    7.0],    
    'Month':           [12.0,  4.0,    7.0,    7.0],    
    'Quarter':         [4.0,   2.0,    3.0,    3.0],    
    'DayofMonth':      [23.0,  12.0,   15.0,   20.0],    
    'DepTime':         [1700.0, 600.0, 1400.0, 2300.0],
    'CRSDepTime':      [1700.0, 600.0, 1400.0, 2300.0],
    'Distance':        [2500.0, 500.0, 1000.0, 800.0],
    'Airline':         [11.0,  16.0,   7.0,    20.0],    
    'OriginStateName': [6.0,   12.0,   32.0,   17.0],   
    'DestStateName':   [36.0,  35.0,   23.0,   23.0],
    'OriginCityName':  [116.0, 21.0,   140.0,  87.0],
    'DestCityName':    [104.0, 135.0,  41.0,   44.0],
    'HourBlock':       [17.0,  6.0,    14.0,   23.0]    
})

predicciones = best_model.predict(casos_prueba)

print("Análisis detallado de predicciones:")

print("\nCaso 1 - Peor escenario (Viernes, temporada alta):")
print(f"Fecha: Viernes 23 de Diciembre")
print(f"Hora: 17:00 (Rush hour)")
print(f"Distancia: 2500 millas (Vuelo largo)")
print(f"Retraso predicho: {format_delay_time(predicciones[0])}")

print("\nCaso 2 - Mejor escenario (Martes temprano):")
print(f"Fecha: Martes 12 de Abril")
print(f"Hora: 06:00")
print(f"Distancia: 500 millas (Vuelo corto)")
print(f"Retraso predicho: {format_delay_time(predicciones[1])}")

print("\nCaso 3 - Caso típico (Mitad de semana):")
print(f"Fecha: Miércoles 15 de Julio")
print(f"Hora: 14:00")
print(f"Distancia: 1000 millas")
print(f"Retraso predicho: {format_delay_time(predicciones[2])}")

print("\nCaso 4 - Último vuelo del día:")
print(f"Fecha: Domingo 20 de Julio")
print(f"Hora: 23:00")
print(f"Distancia: 800 millas")
print(f"Retraso predicho: {format_delay_time(predicciones[3])}")

Análisis detallado de predicciones:

Caso 1 - Peor escenario (Viernes, temporada alta):
Fecha: Viernes 23 de Diciembre
Hora: 17:00 (Rush hour)
Distancia: 2500 millas (Vuelo largo)
Retraso predicho: 2min

Caso 2 - Mejor escenario (Martes temprano):
Fecha: Martes 12 de Abril
Hora: 06:00
Distancia: 500 millas (Vuelo corto)
Retraso predicho: 0min

Caso 3 - Caso típico (Mitad de semana):
Fecha: Miércoles 15 de Julio
Hora: 14:00
Distancia: 1000 millas
Retraso predicho: 3min

Caso 4 - Último vuelo del día:
Fecha: Domingo 20 de Julio
Hora: 23:00
Distancia: 800 millas
Retraso predicho: 5min
